In [1]:
import numpy as np

class Tensor:
  def __init__(self, data, _children=(), _op=''):
    # Allow Parameter objects to pass through (since Parameter inherits from Tensor)
    if hasattr(data, 'data') and hasattr(data, 'grad'):  # It's a Tensor-like object
      self.data = data.data if hasattr(data, 'data') else data
    elif isinstance(data, (int, float, np.ndarray)):
      if isinstance(data, (int, float)):
        self.data = np.array(data)
      else:
        self.data = data
    else:
      raise TypeError(f"Data must be a number or numpy array, got {type(data)}")
    self.grad = np.zeros_like(self.data, dtype=float)  #initialize gradiant
    self._backward = lambda:None
    self._op = _op
    self._prev = set(_children) # Set of input Tensors that created this Tensor
    self.is_parameter = False # Flag for identifying parameters

  def __add__(self, other):
        other = other if isinstance(other, Tensor) else Tensor(other)
        out = Tensor(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad = self.grad + out.grad
            other.grad = other.grad + out.grad
        out._backward = _backward
        return out

  def __mul__(self, other):
    other = other if isinstance(other, Tensor) else Tensor(other)
    out = Tensor(self.data * other.data, (self, other), '*')

    def _backward():
      self.grad = self.grad + other.data * out.grad
      other.grad = other.grad + self.data * out.grad
    out._backward = _backward
    return out

  def __pow__(self, other):
    assert isinstance(other, (int, float)), "only supporting int/float powers for now"
    out = Tensor(self.data**other, (self,), f'**{other}')

    def _backward():
      self.grad = self.grad + (other * self.data**(other-1)) * out.grad
    out._backward = _backward
    return out

  def relu(self):
    out = Tensor(np.maximum(0, self.data), (self,), 'relu')

    def _backward():
      self.grad = self.grad + (out.data > 0) * out.grad
    out._backward = _backward
    return out

  # Basic matrix multiplication for neural networks
  def matmul(self, other):
    other = other if isinstance(other, Tensor) else Tensor(other)
    assert self.data.ndim == 2 and other.data.ndim == 2 and self.data.shape[1] == other.data.shape[0], f"Shape mismatch for matmul: {self.data.shape} @ {other.data.shape}"

    out = Tensor(self.data @ other.data, (self, other), '@')

    def _backward():
      self.grad = self.grad + out.grad @ other.data.T
      other.grad = other.grad + self.data.T @ out.grad
    out._backward = _backward
    return out

  def sum(self, axis=None, keepdims=False):
    out = Tensor(np.sum(self.data, axis=axis, keepdims=keepdims), (self,), 'sum')

    def _backward():
      if axis is not None and not keepdims:
        # Need to expand grad if sum reduced dimensions
        shape_tuple = tuple(1 if i == axis else self.data.shape[i] for i in range(self.data.ndim))
        self.grad = self.grad + np.reshape(out.grad, shape_tuple)
      else:
        self.grad = self.grad + out.grad
    out._backward = _backward
    return out

  def log(self):
    out = Tensor(np.log(self.data), (self,), 'log')

    def _backward():
      epsilon = 1e-8
      self.grad = self.grad + out.grad * (1.0 / (self.data + epsilon))
    out._backward = _backward
    return out

  def tanh(self):
    out = Tensor(np.tanh(self.data), (self,), 'tanh')

    def _backward():
      tanh_val = np.tanh(self.data)
      self.grad = self.grad + out.grad * (1 - tanh_val**2)
    out._backward = _backward
    return out

  def backward(self):
    # Topological sort for correct backpropagation order
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)

    self.grad = np.ones_like(self.data, dtype=float) # Initialize gradient for the output
    for node in reversed(topo):
      node._backward()

  # Enable operations like -x, x-y, /x, x/y
  def __neg__(self): # -self
    return self * -1

  def __sub__(self, other): # self - other
    return self + (-other)

  def __rsub__(self, other): # other - self
    return other + (-self)

  def __truediv__(self, other): # self / other
    return self * (other**-1)

  def __rtruediv__(self, other): # other / self
    return other * (self**-1)

  def __repr__(self):
    return f"Tensor(data={self.data}, grad={self.grad})"

In [2]:
class Parameter(Tensor):
  def __init__(self, data):
    super().__init__(data)
    self.is_parameter = True

In [3]:
class Module(object):
  def __init__(self):
    self._parameters = {}  # Dictionary to hold parameters
    self._modules = {}     # Dictionary to hold sub-modules

  def __setattr__(self, name, value):
    if isinstance(value, Parameter):
      # print(f"  -> Identified as Parameter! Adding to _parameters['{name}']") # Debug print
      self._parameters[name] = value
      super().__setattr__(name, value)
    elif isinstance(value, Module):
      # print(f"  -> Identified as Module! Adding to _modules['{name}']") # Debug print
      self._modules[name] = value
      super().__setattr__(name, value)
    else:
      super().__setattr__(name, value)

  def __call__(self, *args, **kwargs):
    return self.forward(*args, **kwargs)

  def forward(self, *args, **kwargs):
    raise NotImplementedError          # must be implemented by subclasses

  def parameters(self):
    # yields all the parameters of this module and all the sub-modules recursively
    for name, param in self._parameters.items():
      yield param
    for name, module in self._modules.items():
      yield from module.parameters()

  def zero_grad(self):
    # Iterate over the parameters by calling the parameters() method
    for p in self.parameters():
      p.grad = np.zeros_like(p.data, dtype=float)

In [4]:
class Linear(Module):
  def __init__(self, in_features, out_features):
    super().__init__()
    limit = np.sqrt(1/in_features)
    # limit = np.sqrt(6.0 / (in_features + out_features)) # Xavier/Glorot initialization
    self.weight = Parameter(np.random.uniform(-limit, limit, (in_features, out_features)))
    self.bias = Parameter(np.random.uniform(0.0, 0.1, out_features))

  def forward(self, x):
    return x.matmul(self.weight) + self.bias

In [5]:
class ReLU(Module):
  def __init__(self):
    super().__init__()

  def forward(self, x):
    return x.relu()

In [6]:
class Sequential(Module):
  def __init__(self, *modules):
    super().__init__() # Initialize parent Module class
    for i, module in enumerate(modules):
      self._modules[str(i)] = module      # register sub-modules by index

  def forward(self, x):
    for module in self._modules.values():
      x = module(x)
    return x

In [7]:
class SGD():
  def __init__(self, parameters, lr):
    # Collect parameters from the generator into a list
    self.parameters = list(parameters)
    self.lr = lr

  def step(self):
    for p in self.parameters:
      # Ensure parameter has a gradient before updating
      if p.grad is not None:
        p.data = p.data - self.lr * p.grad

  def zero_grad(self):
    for p in self.parameters:
      p.grad = np.zeros_like(p.data, dtype=float)

In [23]:
class Adam():
  def __init__(self, parameters, lr):
    self.parameters = list(parameters)
    self.lr = lr
    self.beta1 = 0.9
    self.beta2 = 0.999
    self.epsilon = 1e-8
    # Initialize m and v as lists of zero arrays, one for each parameter
    self.m = [np.zeros_like(p.data, dtype=float) for p in self.parameters]
    self.v = [np.zeros_like(p.data, dtype=float) for p in self.parameters]
    self.t = 0

  def step(self):
    self.t += 1
    for i, p in enumerate(self.parameters):
      if p.grad is not None:
        # Update biased first and second moment estimates
        self.m[i] = self.beta1 * self.m[i] + ((1 - self.beta1) * p.grad)              # similar like SGD with momentum (not squared)
        self.v[i] = self.beta2 * self.v[i] + ((1 - self.beta2) * (p.grad * p.grad))   # similar like RMSProp (squared)

        # Compute bias-corrected first and second moment estimates
        m_hat = self.m[i] / (1 - (self.beta1 ** self.t))
        v_hat = self.v[i] / (1 - (self.beta2 ** self.t))

        # Update parameters
        p.data = p.data - (self.lr * (m_hat / (np.sqrt(v_hat) + self.epsilon)))

  def zero_grad(self):
    for p in self.parameters:
      p.grad = np.zeros_like(p.data, dtype=float)

In [9]:
def mse_loss(predictions, target):
  return (predictions - target)**2.0

Training Example

In [24]:
# Training data
X_train = Tensor(np.array([[1.0], [2.0], [3.0], [4.0]]))
y_train = Tensor(np.array([[2.0], [4.0], [6.0], [8.0]]))

# Model definition
model = Sequential(Linear(in_features=1, out_features=10), ReLU(), Linear(in_features=10, out_features=1))

# Optimizer
# optimizer = SGD(model.parameters(), lr=0.01)
optimizer = Adam(model.parameters(), lr=0.01)

# Training loop
epochs=100
print("Starting training..")
for epoch in range(epochs):
  # forward pass
  predictions = model(X_train)

  # calculate loss
  loss = mse_loss(predictions, y_train).sum()

  # zero grad
  optimizer.zero_grad()

  # backward pass
  loss.backward()

  # update parameters
  optimizer.step()

  if (epoch + 1) % 10 == 0:
    print(f"Epoch {epoch+1}/{epochs}, Loss {loss.data.item():.4f}")

print("\nTraining complete!")
print("Learned parameters:")
for name, param in model._parameters.items():
  print(f"  {name}: {param.data}")
for name, sub_module in model._modules.items():
  if isinstance(sub_module, Linear):
    print(f"  Linear Layer {name} weights: {sub_module.weight.data}")
    print(f"  Linear Layer {name} biases: {sub_module.bias.data}")

Starting training..
Epoch 10/100, Loss 49.7287
Epoch 20/100, Loss 15.9047
Epoch 30/100, Loss 1.6085
Epoch 40/100, Loss 0.9658
Epoch 50/100, Loss 0.8898
Epoch 60/100, Loss 0.0978
Epoch 70/100, Loss 0.0817
Epoch 80/100, Loss 0.0491
Epoch 90/100, Loss 0.0089
Epoch 100/100, Loss 0.0083

Training complete!
Learned parameters:
  Linear Layer 0 weights: [[-0.38156425 -0.28507449  0.44336106  0.98328343  1.21201436  0.3595418
  -0.28093181 -0.88830835  0.76964661 -0.03508883]]
  Linear Layer 0 biases: [[ 0.02559807  0.07076374  0.09721939  0.12721478  0.00311733 -0.20016814
   0.03460811  0.03739102  0.02270987 -0.18471515]
 [ 0.02559807  0.07076374  0.2270071   0.2513446   0.14073916 -0.23311734
   0.03460811  0.03739102  0.1585589  -0.20774493]
 [ 0.02559807  0.07076374  0.35417915  0.36996814  0.29066992 -0.2467756
   0.03460811  0.03739102  0.30184138 -0.2205614 ]
 [ 0.02559807  0.07076374  0.44597145  0.4554092   0.40097329 -0.25466852
   0.03460811  0.03739102  0.40661243 -0.22671405]]
 